In [23]:
from PIL import Image, ImageDraw
import os

def convert_yolo_annotation(
        annotation_path: str, 
        image_path: str, 
        image_width: int, 
        image_height: int,
        class_mapping=None
    ):
    annotations = []

    with open(annotation_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue

        class_id = int(parts[0])
        center_x, center_y, box_width, box_height = map(float, parts[1:5])

        # Convert normalized coordinates to pixel values
        left = int((center_x - box_width / 2) * image_width)
        top = int((center_y - box_height / 2) * image_height)
        width = int(box_width * image_width)
        height = int(box_height * image_height)

        # Optionally map class IDs to class names
        class_name = class_mapping.get(class_id, str(class_id)) if class_mapping else str(class_id)

        annotations.append({
            'class_id': class_id,
            'class_name': class_name,
            'image_path': image_path,
            'top': top,
            'left': left,
            'width': width,
            'height': height
        })

    return annotations

def write_yolo_annotation(output_path, annotations, image_width, image_height):
    with open(output_path, 'w') as f:
        for annotation in annotations:
            class_id = annotation['class_id']
            top = annotation['top']
            left = annotation['left']
            width = annotation['width']
            height = annotation['height']

            # Convert pixel coordinates to normalized coordinates
            center_x = (left + width / 2) / image_width
            center_y = (top + height / 2) / image_height
            box_width = width / image_width
            box_height = height / image_height

            # Write annotation line in YOLOv5 format
            line = f"{class_id} {center_x:.6f} {center_y:.6f} {box_width:.6f} {box_height:.6f}\n"
            f.write(line)


def create_repetition_image(image_path, annotation_path, image_size: int = 288, number: int = 10):
    # Create a new blank image
    result_image = Image.new("RGB", (image_size, image_size), (255, 255, 255))

    original_image = Image.open(image_path)
    image_width = result_image.width  # Replace with your image width
    image_height = result_image.height  # Replace with your image height

    # Define the number of repetitions and the spacing between pasted regions
    number_repetition = number
    spacing = 5
    x, y = 0, 0  # Initial positions
    current_row_height = 0

    annotations = convert_yolo_annotation(annotation_path, image_path, image_width, image_height)

    new_annotation = []

    for annotation in annotations:
        x1 = annotation['left']
        y1 = annotation['top']
        x2 = x1 + annotation['width']
        y2 = y1 + annotation['height']

        current_row_height = annotation['height'] if annotation['height'] > current_row_height else current_row_height

        cropped_region = original_image.crop((x1, y1, x2, y2))

        for _ in range(number_repetition):

            # If cropped region will be paste exceed result_image then finish
            if y + current_row_height > result_image.height:
                break;
            
            # Paste the cropped region and save the new annotation
            result_image.paste(cropped_region, (x, y))
            new_annotation.append({
                    'class_id': annotation['class_id'],
                    'class_name': annotation['class_name'],
                    'image_path': image_path,
                    'top': y,
                    'left': x,
                    'width': annotation['width'],
                    'height': annotation['height']
                })

            # Move to the next column
            x += cropped_region.width + spacing

            # Check if the next column exceeds the width of the result_image
            if x + cropped_region.width > image_width:
                x = 0  # Reset x to start a new row
                y += current_row_height + spacing  # Move to the next row

    return result_image, new_annotation

In [24]:
# Path template
dataset_path = "datasets/PID_output"
save_dir = "output/images"

# Get a list of all image files in the input folder
image_files_path = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]

cropped_regions = []
cropped_annotations = []

for image_path in image_files_path:
    image_path = os.path.join(dataset_path, image_path)
    annotation_path = image_path.replace('images', 'labels')
    annotation_path = annotation_path.replace('jpg', 'txt')
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    
    original_image = Image.open(image_path)
    annotations = convert_yolo_annotation(annotation_path, image_path, original_image.width, original_image.height)

    for annotation in annotations:
        x1 = annotation['left']
        y1 = annotation['top']
        width = annotation['width']
        height = annotation['height']

        cropped_region = original_image.crop((x1, y1, x1 + width, y1 + height))
        cropped_regions.append(cropped_region)
        cropped_annotations.append(annotation)

print(len(cropped_annotations))
print(len(cropped_regions))

1200
1200


In [137]:
# Create list of images and class_id annotated in that file
dataset_path = "datasets/PID_output"
save_dir = "output/images"

import random
import math
import statistics

def _total_counts(mydataset):
    total_counts = {}

    for index in range(10):
        total_counts[index] = 0
    # Sum the count for each data

    for data in mydataset:
        class_id_counts = data['class_id_counts']
        for key, value in class_id_counts.items():
            total_counts[key] += value

    return total_counts        


def cost_function(total_counts):
    cost = 0
    max_value = 0

    for k, v in total_counts.items():
        max_value = max(v, max_value)

    for k, v in total_counts.items():
        cost += max_value - v

    return cost

# Get a list of all image files in the input folder
image_files_path = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]

mydataset = []

for image_path in image_files_path:
    image_path = os.path.join(dataset_path, image_path)
    annotation_path = image_path.replace('images', 'labels')
    annotation_path = annotation_path.replace('jpg', 'txt')
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    
    original_image = Image.open(image_path)
    annotations = convert_yolo_annotation(annotation_path, image_path, original_image.width, original_image.height)

    class_id_counts = {}

    for index in range(10):
        class_id_counts[index] = 0

    # Count the occureneces of each class_id
    for annotation in annotations:
        class_id = annotation['class_id']
        class_id_counts[class_id] += 1

    mydataset.append({
        'image_path': image_path,
        'annotations': annotations,
        'class_id_counts': class_id_counts,
    })

min_cost = 99999999.0
saved_randomly_list = []
saved_total_number = {}

for _ in range(100000):

    randomly_selected = random.sample(mydataset, 100)

    total_counts = _total_counts(randomly_selected)

    # Create a new dictionary without items with 0 count
    filtered_total_counts = {class_id: count for class_id, count in total_counts.items() if count > 0}

    cost = cost_function(filtered_total_counts)

    if cost < min_cost:
        min_cost = cost
        saved_randomly_list = randomly_selected
        saved_total_number = filtered_total_counts

print(f"Minimum cost function is {min_cost}")
print(saved_total_number)

Minimum cost function is 111
{3: 28, 4: 47, 5: 28, 8: 4, 9: 17}


In [ ]:
for data in saved_randomly_list:
    print(data['image_path'])

In [ ]:
# Separated annotation by class
# classed_annotation = [
#   {'class_id': 0, items: [0, 1, 2, ...]},
#   {'class_id': 1, items: [14, 12, ,,,]},
# ]
def get_classed_index(annotations, target_class_id: int):
    classed_annotations_index = []
    
    return_dict = {}

    for index, annotation in enumerate(annotations):
        if annotation['class_id'] == target_class_id:
            classed_annotations_index.append(index)
        
    return_dict['class_id'] = target_class_id
    return_dict['items'] = classed_annotations_index

    return return_dict

classed_annotations = []

for class_id in range(10):
    index_list = get_classed_index(cropped_annotations, class_id)
    classed_annotations.append(index_list)



In [ ]:
file_index = 0

result_image = Image.new("RGB", (288, 288), (255, 255, 255))

number_repetition = 10
spacing = 5
x, y = 0, 0
current_row_height = 0

new_annotation = []

for index, region in enumerate(cropped_regions):
    annotation = cropped_annotations[index]
    x1 = annotation['left']
    y1 = annotation['top']

    current_row_height = annotation['height'] if annotation['height'] > current_row_height else current_row_height

    for _ in range(number_repetition):

        if y + current_row_height > result_image.height:
            # Save current image and create a new one
            result_image.save(os.path.join(save_dir, f"image_{file_index}.jpg"))
            write_yolo_annotation(os.path.join(save_dir, f"image_{file_index}.txt"), new_annotation, result_image.width, result_image.height)
            result_image = Image.new("RGB", (288, 288), (255, 255, 255))
            file_index += 1
            x, y = 0, 0
            new_annotation = []
        
        # Paste the cropped region and save the new annotation
        result_image.paste(region, (x, y))
        new_annotation.append({
                'class_id': annotation['class_id'],
                'class_name': annotation['class_name'],
                'top': y,
                'left': x,
                'width': annotation['width'],
                'height': annotation['height']
                })
        
        x += region.width + spacing

        if x + region.width > result_image.width:
            x = 0
            y += current_row_height + spacing

result_image.save(os.path.join(save_dir, "image_{file_index}.jpg"))
write_yolo_annotation(os.path.join(save_dir, "image_{file_index}.txt"), new_annotation, result_image.width, result_image.height)